##visualization script for UCE tree
want to present taxonomic and support data on from the UCE analysis

start with the 75% complete matrix

In [110]:
##functions for generating random colors
from ete2 import Tree, faces, TreeStyle, NodeStyle, CircleFace
import colorsys
from numpy import random
from __future__ import division
from collections import OrderedDict

def getPamlPars(ctl):
    #extract arguments froma paml ctl file and stick them into an ordered dictionary
    pardict = OrderedDict()

    for line in ctl.splitlines():
        if line.partition("*")[0]:
            key, value = line.partition("*")[0].strip().split("=")
            #print key.strip(), value
            pardict[key.strip()] = value
    return pardict


def writeCTL(hessian, tree_title, partitions, template_ctl, run = 1):
    if hessian == "hessian":
        template_ctl["usedata"] = "3"
    elif hessian == "post-hessian":
        template_ctl["usedata"] = "2"
    else:
        print "\nneed information for usedata argument\n"
    number_partitions = os.path.basename(partitions).split("_")[0]
    template_ctl["seqfile"] = os.path.basename(partitions)
    template_ctl["treefile"] = tree_title
    n1 = number_partitions + "_partitions"
    n2 = tree_title
    n3 = "_run_{}".format(run)
    template_ctl["outfile"] = "{}_{}_{}.out".format(n1, n2, n3)
    template_ctl["ndata"] = number_partitions
    #print "\nNew file"
    print len(template_ctl.keys())
    ctl_name = "ctl_{}_{}_{}_{}.ctl".format(hes, n3, n1, n2 )

    #helper function to write ctl files, takes a dictionary of parameters to update (pars)
    temp_ctl = r"""
              seed = {seed}
           seqfile = {seqfile}
          treefile = {treefile}
           outfile = {outfile}

             ndata = {ndata}
           seqtype = {seqtype}    * 0: nucleotides; 1:codons; 2:AAs
           usedata = {usedata}    * 0: no data; 1:seq like; 2:normal approximation; 3:out.BV (in.BV)
             clock = {clock}    * 1: global clock; 2: independent rates; 3: correlated rates
           RootAge = {RootAge}  * safe constraint on root age, used if no fossil for root.

             model = {model}    * 0:JC69, 1:K80, 2:F81, 3:F84, 4:HKY85
             alpha = {alpha}    * alpha for gamma rates at sites
             ncatG = {ncatG}   * No. categories in discrete gamma

         cleandata = {cleandata}    * remove sites with ambiguity data (1:yes, 0:no)?

           BDparas = {BDparas}   * birth, death, sampling
       kappa_gamma = {kappa_gamma}      * gamma prior for kappa
       alpha_gamma = {alpha_gamma}      * gamma prior for alpha

       rgene_gamma = {rgene_gamma}    * gammaDir prior for rate for genes
      sigma2_gamma = {sigma2_gamma}  * gammaDir prior for sigma^2     (for clock=2 or 3)

          finetune = {finetune} * auto (0 or 1): times, rates, mixing, paras, RateParas, FossilErr

             print = {print}
            burnin = {burnin}
          sampfreq = {sampfreq}
           nsample = {nsample}

    *** Note: Make your window wider (100 columns) before running the program.
    """
    
    populated_ctl = temp_ctl.format(**template_ctl)
    
    with open(ctl_name, "w") as ff:
            ff.write(populated_ctl) 
            ff.close()
  
    #print populated_ctl



def makeGroups(tree, taxonList, staticColor = None):
    #pass in tree, list of taxa subtending node, return list with mrca of that subclade + all taxa in it, and passes along a user defined color if passed in
    subClade = tree.get_common_ancestor(taxonList)
    return [subClade.nodeid, subClade, staticColor]
###label all nodes of interest 

def random_color(h=None, l=None, s=None):
    if h is None:
        h = random.random()
    if l is None:
        l = random.random()
    if s is None:
        s = random.random()
    return hls2hex(h, l, s)

def rgb2hex(rgb):
    return '#%02x%02x%02x' % rgb

def hls2hex(h, l, s):
    return rgb2hex( tuple(map(lambda x: int(x*255), colorsys.hls_to_rgb(h, l, s))))


def mylayout(node, number_nodes=True, label_support_with_numbers=False):
    #width is set to 2 to increase visibility in large trees when exporting
    node.img_style["hz_line_width"]=4
    node.img_style["vt_line_width"]=4
    #counter2 = 0
    # If node is a leaf, add the nodes name and a its scientific
    # name
    if node.is_leaf():
        node.img_style["size"] = 0
        color = "#000000"
        faces.add_face_to_node( faces.TextFace(node.name, ftype="Arial", fsize=20, fstyle = "italic", fgcolor=color), node, 0 )
        #try adding the family to the tip as well
        if node.name in ranks:
            family = ranks[node.name]
            faces.add_face_to_node( faces.TextFace(family, ftype="Arial", fsize=12, fgcolor="red"), node, 0 )
        if node.up is None:
            node.img_style["size"]=0
            node.dist = 0.35 # you may need to change this value to fit the aspect of your tree
            node.img_style["hz_line_color"] = "#ffffff"
            if UNROOTED:
                node.img_style["vt_line_type"] = 0.001
        
        if node.name in face_dict:
            print "{} in face_dict ".format(node.name)
            faces.add_face_to_node(face_dict[node.name], node, column  = 1, position = "branch-right") #WOuld relly like the image to appear on outside of tre
            #print "found {} matches".format(counter2)
            
            #node.img_style["size"] = 16
            #node.img_style["shape"] = "sphere"
            #node.img_style["fgcolor"] = "#AA0000"

        # Sets the style of leaf nodes
        #node.img_style["size"] = 12
        #node.img_style["shape"] = "circle"
    #If node is an internal node
    else:
        #support_face = CircleFace(radius=100, color="Thistle", style="circle")    
        #support_face.opacity = 0.3
        #faces.add_face_to_node(support_face, node, 0, position="float")
        
        ###make bubble proportional to boostrap support
        # Creates a sphere face whose size is proportional to node's
        # feature "weight"
        #C = CircleFace(radius=node.support, color="Black", style="sphere")
        # Let's make the sphere transparent 
        #C.opacity = 1.0
        # And place as a float face over the tree
        #faces.add_face_to_node(C, node, 0, position="float")
      
        #print "current node id :" + str(node.nodeid)
        #print "is node in BG_COLORS? " + str(node.nodeid) in BG_COLORS
        if node.nodeid in BG_COLORS:
            #print "this node is in colors" + node.nodeid
            node.img_style["bgcolor"] = BG_COLORS[str(node.nodeid)]
            #print BG_COLORS[node.nodeid]
        # Sets the style of internal nodes
        node.img_style["size"] = 0
        #node.img_style["shape"] = "circle"
        #node.img_style["fgcolor"] = "#000000"
        
        num = node.nodenumber
        if number_nodes == True:
            faces.add_face_to_node(faces.TextFace(num, ftype="Arial", fsize=12, fgcolor="purple"), node, 0, position = "branch-right")
        # is the internal node an order?
        if node.order:
            faces.add_face_to_node(faces.TextFace(node.order, ftype="Arial", fsize=25, fgcolor="#black"), node, 0)
        
        if label_support_with_numbers:
            faces.add_face_to_node(faces.TextFace(node.support, ftype="Arial", fsize=10, fgcolor="black"), node, 0, position = "branch-top")
            if node.bootstrap:
                faces.add_face_to_node(faces.TextFace(node.bootstrap, ftype="Arial", fsize=10, fgcolor="grey"), node, 0, position = "branch-bottom")
        makeSupportSymbols(node, node.bootstrap)   

def calibrations_layout(node):
    #width is set to 2 to increase visibility in large trees when exporting
    node.img_style["hz_line_width"]=4
    node.img_style["vt_line_width"]=4
    
    if node.is_leaf():
        node.img_style["size"] = 0
        color = "#000000" # these two lines hide the default node symbol for tips. They are replaced by the textface below
        faces.add_face_to_node( faces.TextFace(node.name, ftype="Arial", fsize=20, fstyle = "italic", fgcolor=color), node, 0 )
        #try adding the family to the tip as well
        if node.name in ranks: # note that ranks is a dictionary that has not been passed in to the function. Could be trouble!
            family = ranks[node.name]
            faces.add_face_to_node( faces.TextFace(family, ftype="Arial", fsize=12, fgcolor="red"), node, 0 )
        if node.up is None:
            node.img_style["size"]=0
            node.dist = 0.35 # you may need to change this value to fit the aspect of your tree
            node.img_style["hz_line_color"] = "#ffffff"
            if UNROOTED:
                node.img_style["vt_line_type"] = 0.001
        #add faces to nodes that have images. face_dict is defined outside this function too....
        if node.name in face_dict:
            faces.add_face_to_node(face_dict[node.name], node, column  = 1, position = "branch-right")

    #If node is an internal node
    else:
        #color the node if is is a major clade
        if node.nodeid in BG_COLORS:
            node.img_style["bgcolor"] = BG_COLORS[str(node.nodeid)]
        # Sets the style of internal nodes
        node.img_style["size"] = 0
        #node.img_style["shape"] = "circle"
        #node.img_style["fgcolor"] = "#000000"
        
        #label internal nodes with nodenumber
        num = node.nodenumber
        faces.add_face_to_node(faces.TextFace(num, ftype="Arial", fsize=16, fgcolor="purple"), node, 0, position = "branch-right")
        # add paml calibrations
        if "paml_cal" in node.features:
            faces.add_face_to_node(faces.TextFace(node.paml_cal, ftype="Arial", fsize=16, fgcolor="black"), node, 0, position = "branch-right")
        if "raw_cal" in node.features:
            faces.add_face_to_node(faces.TextFace(node.raw_cal, ftype="Arial", fsize=16, fgcolor="orange"), node, 0, position = "branch-right")
            


    
    
  #  for node in tt.traverse("postorder"):
   # node.add_features( nodeid = str(uuid.uuid1() ) )
   # node.add_features( nodenumber = str(counter) )# for calibrations
    #node.add_features( order = None)
            
def attachSupport(target_tree, source_tree):
# this function takes the support values from a source tree and puts them on the common nodes of the target tree
    sourceclades = {}
    for node in source_tree.traverse("preorder"):
        if not node.is_leaf():
            kids = (node.get_leaves())
            labels = frozenset([kid.name for kid in kids])            
            support = node.support 
            sourceclades[(labels)] = support
    #now traverse the target tree
    for node in target_tree.traverse("preorder"):
        if not node.is_leaf():
            kids = (node.get_leaves())
            node_hash = frozenset([kid.name for kid in kids])
            node.add_features( bootstrap = sourceclades.get(node_hash) )# for calibrations
            #print node.support, node.bootstrap, node_hash, "\t", len(node_hash), "\n", 
    #return sourceclades
    
def makeSupportSymbols(node, secondary_support):
    #function to make circle with size scaled to base support value and color of circle scaled to the level of the second support value
    opacity = 0.01
    # Let's make the sphere transparent 
    if secondary_support == None:
        circle_color = "Grey"
        opacity = 1.0
        factor = 6
    elif secondary_support >=95.0:
        circle_color = "Black"
        opacity = 1.0
        factor = 10
    elif secondary_support >=70.0:
        circle_color = "Black"
        opacity = 1.0
        factor = 8
    elif secondary_support <70.0:
        circle_color = "Black"
        opacity = 1.0
        factor = 6
    else:
        print "secondary value unexpected: {}".format(secondary_support())
        circle_color = "Red"
        opacity = 1.0
    #circle_color = "#33343E" #a black from the pomacanthus drawing
    #print "color is {}, bootstrap is {}, opacity is {}".format(color, secondary_support, opacity)
    support_face = CircleFace(radius=node.support * factor, color=circle_color, style="circle")    
    support_face.opacity = opacity
    faces.add_face_to_node(support_face, node, 0, position="float-behind")

def addPamlCalibration(lower = None, upper = None, desired_time_scale = 10.0):
    #given an upper and/or lower bound, add a calibration feature to the tree that matches the format needed for mcmc
    #time_scale is the number of MY that 1 time unit will represent in the mcmctree analysis
    #defaults to 1 unit = 10 MY
    if lower:
        scaled_lower = lower / desired_time_scale
    if upper:
        scaled_upper = upper / desired_time_scale
    calibration_str = ""
    """
    if lower and str(lower) != "nan":
        calibration_str += "L({},0.1,1.0,1e-300)".format(lower / desired_time_scale) #for hard lower bound
    if upper and str(upper) != "nan":
        calibration_str += "<{}".format(upper / desired_time_scale )   
    """
    if lower and upper and str(lower) != "nan" and str(upper) != "nan":
        calibration_str = "'B({},{},1e-300,0.05)'".format(scaled_lower, scaled_upper)
    elif not upper and lower and str(lower) != "nan":
    #lower and str(lower) != "nan" and not upper:
        calibration_str = "'L({},0.1,0.5,1e-300)'".format(scaled_lower)
    elif not lower and upper and str(upper) != "nan":
        calibration_str = "'U({}, 0.05)'".format(scaled_upper)
        #print "the cali strin is {}".format(calibration_str)
    return calibration_str


def makePamlTree(treestring):
    #takes a tree, removes NHX annotation but leaves the calibration specification for PAML
    pattern1 = r"\[&&NHX:nodenumber=\d+\]"
    #pattern2 = r"\[&&NHX:nodenumber=\d+:\w+_\w+=([()Le<>\d+.]+)]"
    pattern2 = r"\[&&NHX:nodenumber=\d+:\w+_\w+=('[LB_\d.e-]+')\]"
    pp2 = re.compile(pattern2)
    pattern3 = r"('[LUB])_([\d._e-]+)_(')"
    pp3 = re.compile(pattern3)
   # pattern4
    #print "\n here is the treestring\n", treestring
    temptree = re.sub(pattern1, "", treestring)
    temptree2 = pp2.sub(r"\1", temptree)
    temptree3 = pp3.sub(r"\1(\2)\3", temptree2)
    #temptree4 = 
    finaltree = temptree3.replace("_", ",")
    #print "this is finaltree\n", finaltree
    return finaltree


"""
def annotatePaml(target_tree, scheme):
    # tree to use for annotations and a scheme dictionary with node numbers (matching tree) and lower/upper bounds as values
    for calibration_node_number in scheme:
        
        lower, upper = scheme[calibration_node_number]
        print "node is {} lower is {} upper is {}".format(calibration_node_number, lower, upper)
        calibration_text = addPamlCalibration(lower, upper) #get the calibration string
        print "calibration text is {}".format(calibration_text)
        target_calibration_node = target_tree.search_nodes(nodenumber = str(calibration_node_number))
        #print target[0].nodenumber, calibration_text
        target_calibration_node[0].add_features( paml_cal = calibration_text )
    calibration_tree_nhx = target_tree.write(features = ["nodenumber", "paml_cal"], format = 9,format_root_node=True)
    return makePamlTree(calibration_tree_nhx) #title of the tree?
"""
def annotatePaml(target_tree, scheme, raw = False):
    # tree to use for annotations and a scheme dictionary with node numbers (matching tree) and lower/upper bounds as values
    for calibration_node_number in scheme:
        #print calibration_node_number
        lower, upper = scheme[calibration_node_number]
        print "\n in annotatePaml"
        print "node is {} lower is {} upper is {}\t".format(calibration_node_number, lower, upper)

        if str(lower) == "nan":
            lower = None
        if str(upper) == "nan":
            upper = None
        #print "\n", lower, upper
        

        calibration_text = addPamlCalibration(lower, upper) #get the calibration string
        print "calibration text is {}".format(calibration_text)
        target_calibration_node = target_tree.search_nodes(nodenumber = str(calibration_node_number))
        print "targ cali node is {} and text is {}\n".format(target_calibration_node[0].nodenumber, calibration_text)
        target_calibration_node[0].add_features( paml_cal = str(calibration_text) )
        if raw == True:
            print "raw is true"
            rawtxt = "Lower:{} Upper:{}".format(str(lower), str(upper))
            print "the raw txt is: {}".format(rawtxt)
            target_calibration_node[0].add_features( raw_cal = str(rawtxt) )
            
    calibration_tree_nhx = tt.write(features = ['nodenumber', "raw_cal", "paml_cal"], format = 9,format_root_node=True)
    print calibration_tree_nhx
    return makePamlTree(calibration_tree_nhx) #title of the tree?   


def annotateAndReturn(target_tree, scheme, raw = False):
    # gives the annoated tree in ete2 format
    for calibration_node_number in scheme:
        #print calibration_node_number
        lower, upper = scheme[calibration_node_number]
        print "\n in annotatePaml"
        print "node is {} lower is {} upper is {}\t".format(calibration_node_number, lower, upper)

        if str(lower) == "nan":
            lower = None
        if str(upper) == "nan":
            upper = None
        #print "\n", lower, upper
        

        calibration_text = addPamlCalibration(lower, upper) #get the calibration string
        print "calibration text is {}".format(calibration_text)
        target_calibration_node = target_tree.search_nodes(nodenumber = str(calibration_node_number))
        print "targ cali node is {} and text is {}\n".format(target_calibration_node[0].nodenumber, calibration_text)
        target_calibration_node[0].add_features( paml_cal = str(calibration_text) )
        if raw == True:
            print "raw is true"
            rawtxt = "Lower:{} Upper:{}".format(str(lower), str(upper))
            print "the raw txt is: {}".format(rawtxt)
            target_calibration_node[0].add_features( raw_cal = str(rawtxt) )
            
    #calibration_tree_nhx = tt.write(features = ['nodenumber', "raw_cal", "paml_cal"], format = 9,format_root_node=True)
    return target_tree
    #return makePamlTree(calibration_tree_nhx) #title of the tree?   


def writePamlTree(treestr, title):
    #converts names back to match alignment, prepares a file for mcmctree
    t1 = treestr.lower().replace(" ", "_")
    t2 = t1.replace("takifugu_ocellatus","takifugu_occelatus")
    t3 = t2.replace("coruscum","coruscum2")
    t4 = t3.replace("'b", "'B")
    t4 = t4.replace("'u", "'U")
    t4 = t4.replace("'l", "'L")
    with open(tree_title, 'w') as the_file:
        the_file.write("120 1 \n")
        the_file.write(t4)
        the_file.close()

def copyPamlAlignment(alignpath, writepath):
    shutil.copy(alignpath, writepath)

def writePamlCtl(hessian, tree_title, partitions, template_ctl):
    if hessian == "hessian":
        template_ctl["usedata"] = "3"
    elif hessian == "post-hessian":
        template_ctl["usedata"] = "2"
    else:
        print "\nneed information for usedata argument\n"
    number_partitions = os.path.basename(partitions).split("_")[0]
    template_ctl["seqfile"] = os.path.basename(partitions)
    template_ctl["treefile"] = tree_title
    n1 = number_partitions + "_partitions"
    n2 = tree_title
    template_ctl["outfile"] = "{}_{}.out".format(n1, n2)
    template_ctl["ndata"] = number_partitions
    #print "\nNew file"
    print len(template_ctl.keys())
    ctl_name = "ctl_{}_{}_{}.ctl".format(hes, n1, n2 )
    with open(ctl_name, "w") as ff:
        for key, value in template_ctl.items():
            line = key + " = " + str(value) + "\n"
            ff.writelines(line) 
    ff.close()

Using these tree directories: **/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/trees/75p/** and **/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/trees/75p/**


UPDATE: the tree analyses are finalized. There are (in 75% complete)

- ExaBayes_ConsensusExtendedMajorityRuleNewick.Acanthomorph-75p-STDPART-1.5M-Burn25-FINAL
- ExaBayes_ConsensusExtendedMajorityRuleNewick.Acanthomorph-75p-UNPART-1.5M-Burn25-FINAL.tre
- RAxML.acanthomorph-no-chauliodius-75p-STDPART.tre
- RAxML.acanthomorph-no-chauliodius-75p-UNPART.tre

and in 95% complete:

- ExaBayes_ConsensusExtendedMajorityRuleNewick.Acanthomorph-95p-UNPART-1M-Burn25-FINAL.tre
- RAxML.acanthomorph-no-chauliodius-95p-UNPART.tre

Will present the 75% complete tree in the paper

housekeeping

- set the home directory
- assign identifier to each node for naming purposes
- correct names on tree
- add families
- set the outgroup to Alepisaurus

Taxonomy has already been saved to a file: /Users/michael_alfaro/Dropbox/malfaro-acanthomorph/manuscript/pnas tex/ETE_work/converted_pngs/ranks.txt. Read this in to assign tips to families


In [33]:
#making the acanthomorph tree look good
import os, uuid
from ast import literal_eval
from ete2 import Tree, TreeStyle, AttrFace, NodeStyle

os.chdir("/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/manuscript/pnas tex/ETE_work")
home = "/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/manuscript/pnas tex/ETE_work/ete_acanthomorph_fig/"

exaPartTree = Tree("/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/trees/75p/ExaBayes_ConsensusExtendedMajorityRuleNewick.Acanthomorph-75p-STDPART-1.5M-Burn25-FINAL.tre")
exaNoPartTree = Tree("/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/trees/75p/ExaBayes_ConsensusExtendedMajorityRuleNewick.Acanthomorph-75p-UNPART-1.5M-Burn25-FINAL.tre")
raxPartTree = Tree("/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/trees/75p/RAxML.acanthomorph-no-chauliodius-75p-STDPART.tre")
raxNoPartTree = Tree("/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/trees/75p/RAxML.acanthomorph-no-chauliodius-75p-UNPART.tre")

fourTrees = {
"exa_75_part" : exaPartTree, 
"exa_75_no_part" : exaNoPartTree, 
"rax_75_part" : raxPartTree,  
"rax_75_no_part" : raxNoPartTree 
}

##need to root the exabayes trees, set root branch to 0, assign uuid
for name in fourTrees.keys():
    tt = fourTrees[name]
    ancestor = "alepisaurus_ferox"
    tt.set_outgroup(ancestor)
    tt.get_tree_root().dist = 0.001
    counter = 0
    for node in tt.traverse("postorder"):
        node.add_features( nodeid = str(uuid.uuid1() ) )
        node.add_features( nodenumber = str(counter) )# for calibrations
        node.add_features( order = None)
        counter += 1
        if node.is_leaf():
            node.name = node.name.capitalize().replace("_", " ")
            #print node.name
        if node.name == "Takifugu occelatus":
            node.name = "Takifugu ocellatus"
        if node.name == "Ostorhinchus nigrofasciatus":
            node.name = "Ostorhinchus nigrofasciatus"
        if node.name == "Sargocentron coruscum2":
            node.name = "Sargocentron coruscum"
            print "changed {}".format(node.name)


outfile = "4_27_2015_acanthomorh_renamed.tre"

tt = fourTrees["exa_75_part"]
ml = fourTrees["rax_75_part"] #just grabbing the ml tree for the bs values

os.chdir(home)
tt.write( features = ["nodeid", "nodenumber"], outfile = outfile)


rr = open("/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/manuscript/pnas tex/ETE_work/converted_pngs/ranks.txt").read()
fam_key = literal_eval(rr)
ranks = fam_key

attachSupport(tt, ml)
#print "fam keys has {} keys and {} values".format( len(fam_key.keys() ), len(fam_key.values()))

changed Sargocentron coruscum
changed Sargocentron coruscum
changed Sargocentron coruscum
changed Sargocentron coruscum




Now what? Check to see what images match the illustrations


according to Julie Himes some illustrations are not showing up in the figures:

Sargocentron coruscum, Pholidichthys leucotaenia, Scartella cristata, Cheimarrichthys fosteri, Scarus ferrigineus, Pomacanthus paru



In [122]:
import glob
import os
tips = tt.get_leaf_names() 

png_dir = "/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/manuscript/pnas tex/ETE_work/converted_pngs"
os.chdir(png_dir)
pic_names = glob.glob("*.png") #just get pngs
pic_genus = [gg.split("_")[0] for gg in pic_names]
tip_genus = [names.split(" ")[0] for names in tips]

print "there are {} total illustrations".format(len(pic_names))
for fish in pic_names:
    name =  " ".join(fish.split("_")[0:2])
    intips = names in tips
    print "{} in the tree: {}".format(name, str(intips))

print tips

#set methods to find tip genera that have illustrations
inter = set(tip_genus).intersection(pic_genus)

print inter

there are 48 total illustrations
Acanthurus bahianus in the tree: True
Anoplogaster cornuta in the tree: True
Aphredoderus sayanus in the tree: True
Balistes capriscus in the tree: True
Batrachoides pacifici in the tree: True
Brama japonica in the tree: True
Canthigaster margaritata in the tree: True
Caranx melampygus in the tree: True
Carapus bermudensis in the tree: True
Cheimarrichthys fosteri in the tree: True
Chromis enchrysura in the tree: True
Citharoides macrolepis in the tree: True
Coryphaena hippurus in the tree: True
Coryphaenoides rupestris in the tree: True
Cryptopsaras couesii in the tree: True
Dactyloptena orientalis in the tree: True
Haemulon parra in the tree: True
Hypoplectrus puela in the tree: True
Istiophorus platypterus in the tree: True
Micropterus salmoides in the tree: True
Mogurnda adspersa in the tree: True
Myripristis leiognathus in the tree: True
Naso unicornis in the tree: True
Ogcocephalus radiatus in the tree: True
Ostorhinchus nigrofasciatus in the tree

In [123]:
###make a dict to assign names and imagefaces
from ete2 import Tree, faces, TreeStyle
face_dict ={}
species_dict = {}
for images in glob.glob("*.png"):
    genus = images.split("_")[0]
    species = images.split("_")[1]
    nn = " ".join([genus, species])
    #print nn
    newface = faces.ImgFace(png_dir + "/" + images, width = 175)
    #newface = faces.ImgFace(png_dir + "/" + images)
    face_dict[nn] = newface
#len(face_dict.keys())

#print "there are {} images in face_dict".format(len(face_dict.keys()))

Acanthurus bahianus
Anoplogaster cornuta
Aphredoderus sayanus
Balistes capriscus
Batrachoides pacifici
Brama japonica
Canthigaster margaritata
Caranx melampygus
Carapus bermudensis
Cheimarrichthys fosteri
Chromis enchrysura
Citharoides macrolepis
Coryphaena hippurus
Coryphaenoides rupestris
Cryptopsaras couesii
Dactyloptena orientalis
Haemulon parra
Hypoplectrus puela
Istiophorus platypterus
Micropterus salmoides
Mogurnda adspersa
Myripristis leiognathus
Naso unicornis
Ogcocephalus radiatus
Ostorhinchus nigrofasciatus
Parachirus xenicus
Pareques acuminatus
Parupeneus multifasciatus
Pempheris schomburgkii
Periophthalmus barbarus
Pholidichthys leucotaenia
Polymixia lowei
Pomacanthus paru
Pomatomus saltatrix
Pseudanthias squamipinnis
Pseudochromis flavivertex
Regalecus glesne
Rondeletia loricata
Sargocentron coruscum
Scartella cristata
Scarus ferrigineus
Scomber scombrus
Sphyraena putnamae
Stylephorus chordatus
Syngnathus fuscus
Thalassoma ballieui
Valenciennea strigata
Zeus faber


48

In [124]:
randomcolors = False #flag for selecting random colors or a palette

scarus_palette_complete = ["#ffffff",  "#91c9dd",  "#8ec3da",  "#e4e1da",  "#f1f0ea",  "#d29f7e",  "#db9ab0",  "#efd5db",  "#4f93b9",  "#f0ede6",  "#ede9e7",  "#ec926b",  "#8c7da5",  "#e96e7a",  "#ceaa9f",  "#cc778d",  "#ebe5df",  "#a65d72",  "#b98698",  "#99bbd3",  "#99c5d9",  "#bfd9e2",  "#fef4ee",  "#f0e6dc",  "#ebf3ef",  "#ecebec",  "#f8f4eb",  "#fff6ed"]
scarus_palette_light = ["#d3aeb9", "#c6bed2", "#a7c9dc", "#e6bbc6", "#f6c9b5", "#edcdd8", "#ccdde9"]

clades = {} #node id and taxa of all taxonomic groups 
clades["Paracanthopterygii"] = makeGroups(tt, ("Lampris guttatus", "Gadus morhua"), "#B7E1DC" )
clades["Tetraodontiformes"] = makeGroups(tt, ("Canthigaster rostrata", "Balistes capriscus"), "#E2D2C2" )
#clades["lophiiforms"] = makeGroups(tt, ("Ogcocephalus radiatus", "Antigonia capros") )
clades["Scombrimorpha"] = makeGroups(tt, ("Syngnathus fuscus", "Cubiceps baxteri"), "#DDDCD5" )
clades["Gobiomorpha"] = makeGroups(tt, ("Kurtus gulliveri", "Valenciennea strigata"), "#E2D7D4" )
clades["Beryciformes"] = makeGroups(tt, ("Rondeletia loricata", "Myripristis leiognathus"), "#E6A8A4" )
clades["Ovalentaria"] = makeGroups(tt, ("Xenentodon cancila", "Scartella cristata"), "#D6AC83")
clades["Labridae"] = makeGroups( tt, ("Epibulus insidiator", "Halichoeres poeyi"), "#EEB596" )
clades["Acanthomorpha"] = makeGroups(tt, ("Lampris guttatus", "Canthigaster rostrata"), "#F3E1CD"  )
clades["Syngnathiformes"] = makeGroups( tt, ("Parupeneus multifasciatus", "Syngnathus fuscus"), "#D5C889" )
clades["Scombriformes"]  = makeGroups ( tt, ("Chiasmodon niger", "Cubiceps baxteri") )
#clades["carangids"]  = makeGroups ( tt, ("Alepes kleinii", "Toxotes jaculatrix"), "#EDD6AE" )
clades["Carangimorpha"] = makeGroups( tt, ("Alepes kleinii", "Centropomus medius"), "#F6E0C0")
clades["Pleuronectiformes"] = makeGroups( tt, ("Parachirus xenicus", "Citharoides macrolepis"), "#EBC8AC" )
clades["Percomorpha"] = makeGroups( tt, ("Carapus bermudensis", "Citharoides macrolepis"), "#D8D2C9")
clades["Perciformes"] = makeGroups ( tt, ("Cephalopholis argus", "Taenianotus triacanthus"), "#DECAC4" )
clades["Eupercomorpha"] = makeGroups(tt, ("Canthigaster rostrata", "Haemulon album"), "#D9DED5") 


##add the taxonomic group to the parent node
for clade in clades.keys():
    nid = clades[clade][0]
    res = tt.search_nodes(nodeid = nid)
    res[0].add_features( order = clade)
    #print res[0]


if randomcolors:
    possibleColors = range(360, 0, -int( 360./ len(clades.keys()) ) ) #get a set of color values based on the number of clades
    BG_COLORS= {}
    for clade in clades.keys():
        col = possibleColors.pop()
        color = random_color(h=float(col)/360, s=0.3, l=0.9)
        nodeid = clades[clade][0]
        BG_COLORS[nodeid] = color
    
else:
    BG_COLORS= {}
    for clade in clades.keys():
        nodeid = clades[clade][0]
        if clades[clade][2]:
            #if a user defined color is present
            color = clades[clade][2]
            #print "using user defined color {}".format(color)
        else:
            color = str( random.choice(scarus_palette_light, 1)[0] )
            #print "using random color from palette: {}".format(color)
            
        BG_COLORS[nodeid] = color

    



####Print the calibrations map
label the node numbers. Ideally would save these node numbers as a feature and then use those node numbers to specify the paml file....

In [37]:
I = TreeStyle()
I.layout_fn = mylayout
I.show_leaf_name = False
I.force_topology = True
#I.legend_position = 3
#I.extra_branch_line_type = 1
#I.guiding_lines_type = 1
#I.guiding_lines_color = "#666666"
#I.extra_branch_line_color = "#666666"
I.optimal_scale_level = "semi"
#I.root_opening_factor = 0

writedir = "/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/dating/"

tt.render(writedir +"labelled_nodes_acanthomorphs.pdf", tree_style=I, w=1200)

Anoplogaster cornuta in face_dict 
Regalecus glesne in face_dict 
Polymixia lowei in face_dict 
Rondeletia loricata in face_dict 
Aphredoderus sayanus in face_dict 
Zeus faber in face_dict 
Stylephorus chordatus in face_dict 
Sargocentron coruscum in face_dict 
Carapus bermudensis in face_dict 
Batrachoides pacifici in face_dict 
Coryphaenoides rupestris in face_dict 
Ostorhinchus nigrofasciatus in face_dict 
Syngnathus fuscus in face_dict 
Micropterus salmoides in face_dict 
Dactyloptena orientalis in face_dict 
Scomber scombrus in face_dict 
Hypoplectrus puela in face_dict 
Pholidichthys leucotaenia in face_dict 
Pempheris schomburgkii in face_dict 
Brama japonica in face_dict 
Pseudochromis flavivertex in face_dict 
Cheimarrichthys fosteri in face_dict 
Valenciennea strigata in face_dict 
Scartella cristata in face_dict 
Istiophorus platypterus in face_dict 
Pareques acuminatus in face_dict 
Parachirus xenicus in face_dict 
Pomacanthus paru in face_dict 
Naso unicornis in face_dict 

{'faces': [[702.8425423187474,
   3632.0025550942178,
   795.5924624720531,
   3646.7582242095164,
   197,
   'Naso unicornis'],
  [702.8425423187474,
   3646.7582242095164,
   752.7307569466618,
   3655.1900351325444,
   197,
   'Acanthuridae'],
  [795.5924624720531,
   3603.896518684125,
   918.5563717662083,
   3683.2960715426366,
   197,
   None],
  [523.9859469817944,
   4160.396039603959,
   677.8664963270515,
   4175.151708719258,
   225,
   'Pempheris schomburgkii'],
  [523.9859469817944,
   4175.151708719258,
   573.8741616097088,
   4183.583519642285,
   225,
   'Pempheridae'],
  [677.8664963270515,
   4137.911210475885,
   800.8304056212066,
   4206.068348770359,
   225,
   None],
  [343.0852762695623,
   1010.7633343979555,
   352.2197381028424,
   1019.1951453209833,
   37,
   '33'],
  [657.6173746406893,
   3336.2304375598837,
   671.6703928457356,
   3344.6622484829118,
   182,
   '173'],
  [206.51549025870324,
   1023.9329220806204,
   220.56850846374954,
   1032.364733

###writing files for mcmctree analysis

I have three schemes to evaluate and it is a pain to create the hessian and post-hessian files for each scheme for each alignment. I am writing a block to populate the files needed for the analysis. What needs to happens:

- for each alignment (partitioned or unpartitioned)
    - for each scheme
        - make a directory with 
            - alignment
            - treefile
            - control file
        - modify the control file
            - hessian or post-hessian
            - outfile name
            - treefile name
            - use data
            - number of genes

making run files for paml analysis using 95% complete matrix

In [38]:
import os
import glob
import shutil
import pandas as pd
from ete2 import TextFace

##Read in the calibrations file
import pandas as pd
import re as re
infile = "/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/dating/calibration_bounds.csv"
writedir = "/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/dating/"
alignpath = "/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/dating/alignments/"
outpath = "/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/junk"
dd = pd.read_csv(infile, index_col=0, na_values="n/a") #the nodes are the index in this dataframe
pd.set_option('float_format', '{:20,.2f}'.format)
scheme1, scheme2, scheme3 = {}, {}, {} #dictionaries to hold lower and upper bounds
heshlist = ["hessian", "post-hessian"]
nruns = 2 #number of runs for each analysis using the approximation

##for making the control file
ctlfile = "/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/dating/control_file_template/mcmc.ctl"
###should move this elsewhere--only need to do it once to get the paml ctl structure
with open(ctlfile, 'r') as ff:
    ctl = ff.read()
    ff.close()




###overwrites files eachtime this is called
if not os.path.exists(outpath):
    os.makedirs(outpath)
else:
    shutil.rmtree(outpath)           #removes all the subdirectories!
    os.makedirs(outpath)
    
#populate the scheme dictionaries
for cal_node in dd.index:
    min_age = dd.loc[cal_node][1] 
    max_age_1 = dd.loc[cal_node][2]
    max_age_2 = dd.loc[cal_node][3]
    max_age_3 = dd.loc[cal_node][4]
    
    scheme1[cal_node] = [min_age, max_age_1]
    scheme2[cal_node] = [min_age, max_age_2]
    scheme3[cal_node] = [min_age, max_age_3]
    
    

In [39]:
schemes = [scheme1, scheme2, scheme3]
aligns = glob.glob(alignpath + "*.seq")
basectl = getPamlPars(ctl)


    

#basectl['RootAge'] = "'B(12.50, 15.49, 1e-300, 0.05)'" 
basectl['RootAge'] = "''" 
for alignment in aligns:
    aligndir = outpath + os.path.basename(alignment).split("_")[0] + "_part/"
    for num, scheme in enumerate(schemes):
        tree_title = "scheme_{}.tre".format(num)

        for hes in heshlist:
            if hes == "post-hessian":
                for run in range(nruns):
                    schemenum = str(num)
                    curdir = aligndir + "scheme_{}_{}_run_{}".format(num, hes, run+1)
                    os.makedirs(curdir)
                    os.chdir(curdir)
                    paml_tree_string = annotatePaml(tt, scheme)
                    
                    writePamlTree(paml_tree_string, tree_title )
                    copyPamlAlignment(alignment, curdir)
                    #writePamlCtl(hes, tree_title, alignment, basectl)
                    writeCTL(hes, tree_title, alignment, basectl, run+1, )
            else:
                schemenum = str(num)
                curdir = aligndir + "scheme_{}_{}".format(num, hes)
                os.makedirs(curdir)
                os.chdir(curdir)
                #tree_title = "scheme_{}.tre".format(num)
                writePamlTree(paml_tree_string, tree_title )
                copyPamlAlignment(alignment, curdir)
                #writePamlCtl(hes, tree_title, alignment, basectl)
                writeCTL(hes, tree_title, alignment, basectl)
       

OSError: [Errno 17] File exists: '/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/junk01_part/scheme_0_hessian'

In [88]:
annotated_tree = annotateAndReturn(tt, scheme3, raw = True)
calibration_tree_nhx = annotated_tree.write(features = ['nodenumber', "paml_cal"], format = 9,format_root_node=True)
paml_tree_string = makePamlTree(calibration_tree_nhx)
title = "no_outs.tre"
writePamlTree(paml_tree_string, title )
                    #copyPamlAlignment(alignment, curdir)
                    #writePamlCtl(hes, tree_title, alignment, basectl)
                    #writeCTL(hes, tree_title, alignment, basectl, run+1, )


 in annotatePaml
node is 130 lower is 41.2 upper is 66.6	
calibration text is 'B(4.12,6.66,1e-300,0.05)'
targ cali node is 130 and text is 'B(4.12,6.66,1e-300,0.05)'

raw is true
the raw txt is: Lower:41.2 Upper:66.6

 in annotatePaml
node is 132 lower is 49.0 upper is 79.7	
calibration text is 'B(4.9,7.97,1e-300,0.05)'
targ cali node is 132 and text is 'B(4.9,7.97,1e-300,0.05)'

raw is true
the raw txt is: Lower:49.0 Upper:79.7

 in annotatePaml
node is 150 lower is 49.0 upper is 69.3	
calibration text is 'B(4.9,6.93,1e-300,0.05)'
targ cali node is 150 and text is 'B(4.9,6.93,1e-300,0.05)'

raw is true
the raw txt is: Lower:49.0 Upper:69.3

 in annotatePaml
node is 6 lower is 54.17 upper is 122.5	
calibration text is 'B(5.417,12.25,1e-300,0.05)'
targ cali node is 6 and text is 'B(5.417,12.25,1e-300,0.05)'

raw is true
the raw txt is: Lower:54.17 Upper:122.5

 in annotatePaml
node is 137 lower is 55.2 upper is 94.6	
calibration text is 'B(5.52,9.46,1e-300,0.05)'
targ cali node is 137 

In [59]:
%qtconsole

In [58]:
%edit


#get tree string 


 in annotatePaml
node is 130 lower is 41.2 upper is 66.6	
calibration text is 'B(4.12,6.66,1e-300,0.05)'
targ cali node is 130 and text is 'B(4.12,6.66,1e-300,0.05)'

raw is true
the raw txt is: Lower:41.2 Upper:66.6

 in annotatePaml
node is 132 lower is 49.0 upper is 79.7	
calibration text is 'B(4.9,7.97,1e-300,0.05)'
targ cali node is 132 and text is 'B(4.9,7.97,1e-300,0.05)'

raw is true
the raw txt is: Lower:49.0 Upper:79.7

 in annotatePaml
node is 150 lower is 49.0 upper is 69.3	
calibration text is 'B(4.9,6.93,1e-300,0.05)'
targ cali node is 150 and text is 'B(4.9,6.93,1e-300,0.05)'

raw is true
the raw txt is: Lower:49.0 Upper:69.3

 in annotatePaml
node is 6 lower is 54.17 upper is 122.5	
calibration text is 'B(5.417,12.25,1e-300,0.05)'
targ cali node is 6 and text is 'B(5.417,12.25,1e-300,0.05)'

raw is true
the raw txt is: Lower:54.17 Upper:122.5

 in annotatePaml
node is 137 lower is 55.2 upper is 94.6	
calibration text is 'B(5.52,9.46,1e-300,0.05)'
targ cali node is 137 

TypeError: expected string or buffer

In [45]:


#taxa_to_keep.remove("Alepisaurus ferox")
#taxa_to_keep.remove("Ceratoscopelus warmingii")



#prune the outgroup species and make a PAML tree and control_file set

118

 in annotatePaml
node is 130 lower is 41.2 upper is 66.6	
calibration text is 'B(4.12,6.66,1e-300,0.05)'


AttributeError: 'NoneType' object has no attribute 'search_nodes'

In [43]:
       
#just print out scheme 3
J = TreeStyle()
J.layout_fn = calibrations_layout
J.show_leaf_name = False
J.force_topology = True
#I.legend_position = 3
J.optimal_scale_level = "semi"


scheme = schemes[2]
tree_title = "scheme_{}.tre".format(2)
paml_tree_string = annotatePaml(tt, scheme, raw = True)
writePamlTree(paml_tree_string, tree_title )
J.title.add_face(TextFace(tree_title, fsize=20), column=0)
tt.render(writedir +"scheme_2_labelled_pamlCalibrations_acanthomorphs_{}.pdf".format(tree_title), tree_style=J, w=1200)

            
           


 in annotatePaml
node is 130 lower is 41.2 upper is 66.6	
calibration text is 'B(4.12,6.66,1e-300,0.05)'
targ cali node is 130 and text is 'B(4.12,6.66,1e-300,0.05)'

raw is true
the raw txt is: Lower:41.2 Upper:66.6

 in annotatePaml
node is 132 lower is 49.0 upper is 79.7	
calibration text is 'B(4.9,7.97,1e-300,0.05)'
targ cali node is 132 and text is 'B(4.9,7.97,1e-300,0.05)'

raw is true
the raw txt is: Lower:49.0 Upper:79.7

 in annotatePaml
node is 150 lower is 49.0 upper is 69.3	
calibration text is 'B(4.9,6.93,1e-300,0.05)'
targ cali node is 150 and text is 'B(4.9,6.93,1e-300,0.05)'

raw is true
the raw txt is: Lower:49.0 Upper:69.3

 in annotatePaml
node is 6 lower is 54.17 upper is 122.5	
calibration text is 'B(5.417,12.25,1e-300,0.05)'
targ cali node is 6 and text is 'B(5.417,12.25,1e-300,0.05)'

raw is true
the raw txt is: Lower:54.17 Upper:122.5

 in annotatePaml
node is 137 lower is 55.2 upper is 94.6	
calibration text is 'B(5.52,9.46,1e-300,0.05)'
targ cali node is 137 

{'faces': [[825.8278145695365,
   3379.4701986754962,
   913.2450331125829,
   3393.377483443708,
   193,
   'Naso unicornis'],
  [825.8278145695365,
   3393.377483443708,
   872.8476821192054,
   3401.324503311258,
   193,
   'Acanthuridae'],
  [913.2450331125829,
   3352.980132450331,
   1029.1390728476822,
   3427.8145695364237,
   193,
   None],
  [538.4105960264901,
   3877.483443708609,
   683.4437086092715,
   3891.3907284768206,
   221,
   'Pempheris schomburgkii'],
  [538.4105960264901,
   3891.3907284768206,
   585.430463576159,
   3899.3377483443705,
   221,
   'Pempheridae'],
  [683.4437086092715,
   3856.2913907284765,
   799.3377483443709,
   3920.529801324503,
   221,
   None],
  [326.4900662251656,
   907.2847682119204,
   338.4105960264901,
   918.5430463576158,
   33,
   '33'],
  [564.9006622516557,
   3099.048013245033,
   582.7814569536424,
   3110.3062913907283,
   178,
   '173'],
  [273.50993377483445,
   919.6971640681588,
   291.3907284768212,
   930.95544221385

In [3]:
import os
import glob
import shutil
import pandas as pd

##Read in the calibrations file
import pandas as pd
import re as re
infile = "/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/dating/calibration_bounds.csv"
writedir = "/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/dating/"
dd = pd.read_csv(infile, index_col=0, na_values="n/a") #the nodes are the index in this dataframe
pd.set_option('float_format', '{:20,.2f}'.format)
scheme1, scheme2, scheme3 = {}, {}, {} #dictionaries to hold lower and upper bounds


##for making the control file
ctlfile = "/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/dating/control_file_template/mcmc.ctl"
###should move this elsewhere--only need to do it once to get the paml ctl structure
with open(ctlfile, 'r') as ff:
    ctl = ff.read()
    ff.close()



outpath = "/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/dating/95_comp_mcmctree_runfiles/"
###overwrites files eachtime this is called
if not os.path.exists(outpath):
    os.makedirs(outpath)
else:
    shutil.rmtree(outpath)           #removes all the subdirectories!
    os.makedirs(outpath)

#partitioned_seqs = "sate-gblocks-clean-min-90-taxa.PAML.phylip.seq"
#singlepart_seqs = "ACANTHOMORPH-sate-gblocks-clean-min-90-taxa-CONCAT.txt.seq"



alignpath = "/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/dating/alignments/"
writedir = "/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/dating/"
dd = pd.read_csv(infile, index_col=0, na_values="n/a") #the nodes are the index in this dataframe
pd.set_option('float_format', '{:20,.2f}'.format)
scheme1, scheme2, scheme3 = {}, {}, {} #dictionaries to hold lower and upper bounds
heshlist = ["hessian", "post-hessian"]
nruns = 2 #number of runs for each analysis using the approximation



#populate the scheme dictionaries
for cal_node in dd.index:
    min_age = dd.loc[cal_node][1] 
    max_age_1 = dd.loc[cal_node][2]
    max_age_2 = dd.loc[cal_node][3]
    max_age_3 = dd.loc[cal_node][4]
    
    scheme1[cal_node] = [min_age, max_age_1]
    scheme2[cal_node] = [min_age, max_age_2]
    scheme3[cal_node] = [min_age, max_age_3]

schemes = [scheme1, scheme2, scheme3]
aligns = glob.glob(alignpath + "*.seq")
basectl = getPamlPars(ctl)
basectl['RootAge'] = "'B(12.50, 15.49, 1e-300, 0.05)'" 
for alignment in aligns:
    aligndir = outpath + os.path.basename(alignment).split("_")[0] + "_part/"
    for num, scheme in enumerate(schemes):

        for hes in heshlist:
            if hes == "post-hessian":
                for run in range(nruns):
                    schemenum = str(num)
                    curdir = aligndir + "scheme_{}_{}_run_{}".format(num, hes, run+1)
                    os.makedirs(curdir)
                    os.chdir(curdir)
                    paml_tree_string = annotatePaml(tt, scheme)
                    tree_title = "scheme_{}.tre".format(num)
                    writePamlTree(paml_tree_string, tree_title )
                    copyPamlAlignment(alignment, curdir)
                    #writePamlCtl(hes, tree_title, alignment, basectl)
                    writeCTL(hes, tree_title, alignment, run+1, basectl)
            else:
                schemenum = str(num)
                curdir = aligndir + "scheme_{}_{}".format(num, hes)
                os.makedirs(curdir)
                os.chdir(curdir)
                paml_tree_string = annotatePaml(tt, scheme)
                tree_title = "scheme_{}.tre".format(num)
                writePamlTree(paml_tree_string, tree_title )
                copyPamlAlignment(alignment, curdir)
                #writePamlCtl(hes, tree_title, alignment, basectl)
                writeCTL(hes, tree_title, alignment, run+1, basectl)
            
           

 

            

    




    

seed  -1
seqfile  ACANTHOMORPH-sate-gblocks-clean-min-90-taxa-CONCAT.txt
treefile  scheme_0.tre
outfile  scheme_0_unpart_mult_cali_run_1.out
ndata  1
seqtype  0
usedata  2
clock  2
RootAge  '>12.50<15.49'
model  4
alpha  0.5
ncatG  5
cleandata  0
BDparas  0.1 0.1 0.01
kappa_gamma  6 2
alpha_gamma  1 1
rgene_gamma  2 20.372 1
sigma2_gamma  1 100 1
finetune  1: .1 .1 .1 .1 .1 .1
print  1
burnin  10000
sampfreq  20
nsample  15000

41.2 None

49.0 None

49.0 None

54.17 None

55.2 None

58.551 None

93.9 None

49.0 None

32.02 None

53.7 None

69.71 None

98.0 158.3

54.17 None

49.0 None

5.33 None

29.62 None

49.0 None

54.17 None

49.0 None

69.71 None

32.02 56.0

50.5 None

54.17 None

11.9 None

45.46 100.5

49.0 130.8

93.9 None

125.0 154.9

49.0 None
this is finaltree
(Alepisaurus ferox,(Ceratoscopelus warmingii,(((Lampris guttatus,(Zu elongatus,Regalecus glesne)'L(0.533,0.1,0.5,1e-300)')'L(5.417,0.1,0.5,1e-300)',((Polymixia lowei,(Percopsis omiscomycus,Aphredoderus sayanus)'L(5.

NameError: name 'run' is not defined

In [29]:
len(basectl.keys())

23

In [12]:
from ete2 import TextFace

##Read in the calibrations file
import pandas as pd
import re as re
infile = "/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/dating/calibration_bounds.csv"
writedir = "/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/dating/"
dd = pd.read_csv(infile, index_col=0, na_values="n/a") #the nodes are the index in this dataframe
pd.set_option('float_format', '{:20,.2f}'.format)
scheme1, scheme2, scheme3 = {}, {}, {} #dictionaries to hold lower and upper bounds

#populate the scheme dictionaries
for cal_node in dd.index:
    min_age = dd.loc[cal_node][1] 
    max_age_1 = dd.loc[cal_node][2]
    max_age_2 = dd.loc[cal_node][3]
    max_age_3 = dd.loc[cal_node][4]
    
    scheme1[cal_node] = [min_age, max_age_1]
    scheme2[cal_node] = [min_age, max_age_2]
    scheme3[cal_node] = [min_age, max_age_3]

schemes = [scheme1, scheme2, scheme3]

J = TreeStyle()
J.layout_fn = calibrations_layout
J.show_leaf_name = False
J.force_topology = True
#I.legend_position = 3
J.optimal_scale_level = "semi"

counter = 0           
for scheme in schemes:
    #this loop converts the names in the paml tree back to match the alignment
    paml_tree_string = annotatePaml(tt, scheme)
    print paml_tree_string
    t1 = paml_tree_string.lower().replace(" ", "_")
    t2 = t1.replace("takifugu_ocellatus","takifugu_occelatus")
    t3 = t2.replace("coruscum","coruscum2")
    tree_title = "scheme_{}.tre".format(counter)
    with open(writedir + tree_title, 'w') as the_file:
        the_file.write("120 1 \n")
        the_file.write(t3)
        the_file.close()
    counter +=1
    J.title.add_face(TextFace(tree_title, fsize=20), column=0)
    tt.render(writedir +"labelled_pamlCalibrations_acanthomorphs_{}.pdf".format(tree_title), tree_style=J, w=1200)
    



###want to write ctl file to a base directory for hessian and posthessian for all schemes...











41.2 None

49.0 None

49.0 None

54.17 None

55.2 None

58.551 None

93.9 None

49.0 None

32.02 None

53.7 None

69.71 None

98.0 158.3

54.17 None

49.0 None

5.33 None

29.62 None

49.0 None

54.17 None

49.0 None

69.71 None

32.02 56.0

50.5 None

54.17 None

11.9 None

45.46 100.5

49.0 130.8

93.9 None

49.0 None
this is finaltree
(Alepisaurus ferox,(Ceratoscopelus warmingii,(((Lampris guttatus,(Zu elongatus,Regalecus glesne)'L(0.533,0.1,0.5,1e-300)')'L(5.417,0.1,0.5,1e-300)',((Polymixia lowei,(Percopsis omiscomycus,Aphredoderus sayanus)'L(5.8551,0.1,0.5,1e-300)')'L(9.39,0.1,0.5,1e-300)',((Zeus faber,Zenopsis conchifera)'L(3.202,0.1,0.5,1e-300)',(Stylephorus chordatus,(Coryphaenoides rupestris,Gadus morhua)'L(5.37,0.1,0.5,1e-300)'))'L(6.971,0.1,0.5,1e-300)'))'B(9.8,15.83,1e-300,0.05)',(Anoplogaster cornuta,((Rondeletia loricata,(Sargocentron coruscum,(Myripristis violacea,Myripristis leiognathus))'L(4.9,0.1,0.5,1e-300)'),((Carapus bermudensis,Lepophidium profundorum),(Batrachoi

In [13]:
#schemes
paml_tree_string = annotatePaml(tt, schemes[0])


41.2 None

49.0 None

49.0 None

54.17 None

55.2 None

58.551 None

93.9 None

49.0 None

32.02 None

53.7 None

69.71 None

98.0 158.3

54.17 None

49.0 None

5.33 None

29.62 None

49.0 None

54.17 None

49.0 None

69.71 None

32.02 56.0

50.5 None

54.17 None

11.9 None

45.46 100.5

49.0 130.8

93.9 None

49.0 None
this is finaltree
(Alepisaurus ferox,(Ceratoscopelus warmingii,(((Lampris guttatus,(Zu elongatus,Regalecus glesne)'L(0.533,0.1,0.5,1e-300)')'L(5.417,0.1,0.5,1e-300)',((Polymixia lowei,(Percopsis omiscomycus,Aphredoderus sayanus)'L(5.8551,0.1,0.5,1e-300)')'L(9.39,0.1,0.5,1e-300)',((Zeus faber,Zenopsis conchifera)'L(3.202,0.1,0.5,1e-300)',(Stylephorus chordatus,(Coryphaenoides rupestris,Gadus morhua)'L(5.37,0.1,0.5,1e-300)'))'L(6.971,0.1,0.5,1e-300)'))'B(9.8,15.83,1e-300,0.05)',(Anoplogaster cornuta,((Rondeletia loricata,(Sargocentron coruscum,(Myripristis violacea,Myripristis leiognathus))'L(4.9,0.1,0.5,1e-300)'),((Carapus bermudensis,Lepophidium profundorum),(Batrachoi

In [14]:
t1 = paml_tree_string.lower().replace(" ", "_")
t2 = t1.replace("takifugu_ocellatus","takifugu_occelatus")
t3 = t2.replace("coruscum","coruscum2")



In [125]:
I = TreeStyle()
I.tree_width = 1200
I.layout_fn = mylayout
I.show_branch_length = False
#I.show_branch_support = True
I.show_leaf_name = False
I.mode = "c"
I.force_topology = True
#I.legend_position = 3
I.extra_branch_line_type = 1
I.guiding_lines_type = 1
I.guiding_lines_color = "#666666"
I.extra_branch_line_color = "#666666"
I.optimal_scale_level = "full"
I.root_opening_factor = 0

tt.ladderize()
tt.render("treeimg_2a.pdf", tree_style=I, w=1200)

I = TreeStyle()
I.layout_fn = mylayout
I.show_leaf_name = False
I.force_topology = True
#I.legend_position = 3
#I.extra_branch_line_type = 1
#I.guiding_lines_type = 1
#I.guiding_lines_color = "#666666"
#I.extra_branch_line_color = "#666666"
I.optimal_scale_level = "semi"
#I.root_opening_factor = 0

tt.render("treeimg_3a.pdf", tree_style=I, w=1200)


Anoplogaster cornuta in face_dict 
Regalecus glesne in face_dict 
Polymixia lowei in face_dict 
Rondeletia loricata in face_dict 
Aphredoderus sayanus in face_dict 
Zeus faber in face_dict 
Stylephorus chordatus in face_dict 
Sargocentron coruscum in face_dict 
Carapus bermudensis in face_dict 
Batrachoides pacifici in face_dict 
Coryphaenoides rupestris in face_dict 
Myripristis leiognathus in face_dict 
Ostorhinchus nigrofasciatus in face_dict 
Syngnathus fuscus in face_dict 
Micropterus salmoides in face_dict 
Mogurnda adspersa in face_dict 
Dactyloptena orientalis in face_dict 
Scomber scombrus in face_dict 
Sphyraena putnamae in face_dict 
Hypoplectrus puela in face_dict 
Parupeneus multifasciatus in face_dict 
Pholidichthys leucotaenia in face_dict 
Chromis enchrysura in face_dict 
Pseudanthias squamipinnis in face_dict 
Pempheris schomburgkii in face_dict 
Brama japonica in face_dict 
Pomatomus saltatrix in face_dict 
Pseudochromis flavivertex in face_dict 
Citharoides macrolepi

{'faces': [[686.4376130198915,
   4368.89692585895,
   781.9168173598554,
   4384.086799276672,
   208,
   'Naso unicornis'],
  [686.4376130198915,
   4384.086799276672,
   737.7938517179024,
   4392.766726943942,
   208,
   'Acanthuridae'],
  [781.9168173598554,
   4339.963833634719,
   908.499095840868,
   4421.699819168173,
   208,
   None],
  [500.5424954792043,
   3622.4231464737786,
   658.9511754068716,
   3637.6130198915002,
   174,
   'Pempheris schomburgkii'],
  [500.5424954792043,
   3637.6130198915002,
   551.8987341772151,
   3646.29294755877,
   174,
   'Pempheridae'],
  [658.9511754068716,
   3599.2766726943937,
   785.5334538878843,
   3669.439421338155,
   174,
   None],
  [309.5840867992767,
   1031.1030741410486,
   318.9873417721519,
   1039.7830018083182,
   33,
   '33'],
  [638.6980108499096,
   4064.4213381555146,
   653.1645569620254,
   4073.101265822784,
   193,
   '173'],
  [167.8119349005425,
   1035.1214141569774,
   182.27848101265823,
   1043.801341824247

###integrating figures into the acanthomorph manuscript

The directory for the latex figure is here: **/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/manuscript/pnas tex/figures/**


In [126]:
fig_dir = "/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/manuscript/pnas tex/figures/"

I = TreeStyle()
I.layout_fn = mylayout
I.show_leaf_name = False
I.force_topology = True
#I.legend_position = 3
#I.extra_branch_line_type = 1
#I.guiding_lines_type = 1
#I.guiding_lines_color = "#666666"
#I.extra_branch_line_color = "#666666"
I.optimal_scale_level = "semi"
#I.root_opening_factor = 0
#I.show_branch_support = True

tree_name = "bayes_75per_comb_right.pdf"
file_name = fig_dir + tree_name
print file_name

tt.render(file_name, tree_style=I, w=1200)

/Users/michael_alfaro/Dropbox/malfaro-acanthomorph/manuscript/pnas tex/figures/bayes_75per_comb_right.pdf
Anoplogaster cornuta in face_dict 
Regalecus glesne in face_dict 
Polymixia lowei in face_dict 
Rondeletia loricata in face_dict 
Aphredoderus sayanus in face_dict 
Zeus faber in face_dict 
Stylephorus chordatus in face_dict 
Sargocentron coruscum in face_dict 
Carapus bermudensis in face_dict 
Batrachoides pacifici in face_dict 
Coryphaenoides rupestris in face_dict 
Myripristis leiognathus in face_dict 
Ostorhinchus nigrofasciatus in face_dict 
Syngnathus fuscus in face_dict 
Micropterus salmoides in face_dict 
Mogurnda adspersa in face_dict 
Dactyloptena orientalis in face_dict 
Scomber scombrus in face_dict 
Sphyraena putnamae in face_dict 
Hypoplectrus puela in face_dict 
Parupeneus multifasciatus in face_dict 
Pholidichthys leucotaenia in face_dict 
Chromis enchrysura in face_dict 
Pseudanthias squamipinnis in face_dict 
Pempheris schomburgkii in face_dict 
Brama japonica in 

{'faces': [[686.4376130198915,
   4368.89692585895,
   781.9168173598554,
   4384.086799276672,
   208,
   'Naso unicornis'],
  [686.4376130198915,
   4384.086799276672,
   737.7938517179024,
   4392.766726943942,
   208,
   'Acanthuridae'],
  [781.9168173598554,
   4339.963833634719,
   908.499095840868,
   4421.699819168173,
   208,
   None],
  [500.5424954792043,
   3622.4231464737786,
   658.9511754068716,
   3637.6130198915002,
   174,
   'Pempheris schomburgkii'],
  [500.5424954792043,
   3637.6130198915002,
   551.8987341772151,
   3646.29294755877,
   174,
   'Pempheridae'],
  [658.9511754068716,
   3599.2766726943937,
   785.5334538878843,
   3669.439421338155,
   174,
   None],
  [309.5840867992767,
   1031.1030741410486,
   318.9873417721519,
   1039.7830018083182,
   33,
   '33'],
  [638.6980108499096,
   4064.4213381555146,
   653.1645569620254,
   4073.101265822784,
   193,
   '173'],
  [167.8119349005425,
   1035.1214141569774,
   182.27848101265823,
   1043.801341824247

In [ ]:
os.getcwd()

In [18]:
bigtree = Tree("/Users/michael_alfaro/Downloads/PhylogeneticResources/Vascular_Plants_rooted.dated.tre")

In [29]:
tips = bigtree.get_leaf_names()

In [31]:
dd = open("/Users/michael_alfaro/Dropbox/plant_physiology/species_in_data.txt").readlines()

In [ ]:
print len(dd)
presenttips = []
for sp in dd:
    if sp.strip() in tips:
        presenttips.append(sp.strip())
        
print "there are {} from the data  in the tree".format(len(presenttips))  

taxa_to_drop = list(set(tips) - set(presenttips))
print len(taxa_to_drop)
bigtree.prune(taxa_to_drop)
bigtree.describe()
        
        
        
        
        
        
    
    
    

In [ ]:
print "hi"

In [16]:
I = TreeStyle()
I.layout_fn = mylayout
I.show_leaf_name = False
I.force_topology = True
#I.legend_position = 3
#I.extra_branch_line_type = 1
#I.guiding_lines_type = 1
#I.guiding_lines_color = "#666666"
#I.extra_branch_line_color = "#666666"
I.optimal_scale_level = "semi"
#I.root_opening_factor = 0
#I.show_branch_support = True

bigtree.render("bigtree.tre", w=1200)

{'faces': [[546.5741288966587,
   75685.0858672957,
   759.7281490176435,
   75710.7670745392,
   5907,
   'Caulolatilus_affinis'],
  [578.2497954413788,
   73579.22687332935,
   793.9719362867127,
   73604.90808057283,
   5748,
   'Chaetodon_selene'],
  [555.2067594553702,
   75710.7670745392,
   837.7000391337837,
   75736.44828178268,
   5910,
   'Malacanthus_latovittatus'],
  [555.2067594553702,
   75736.44828178268,
   804.3144697172439,
   75762.12948902618,
   5911,
   'Malacanthus_plumieri'],
  [547.502398490534,
   75762.12948902618,
   832.5637988932967,
   75787.81069626966,
   5912,
   'Malacanthus_brevirostris'],
  [546.2941869726086,
   75813.49190351316,
   756.8800863692443,
   75839.17311075664,
   5917,
   'Callanthias_legras'],
  [538.5898247995609,
   75787.81069626966,
   738.9032412987998,
   75813.49190351316,
   5915,
   'Callanthias_ruber'],
  [546.2941869726086,
   75839.17311075664,
   782.5612936127364,
   75864.85431800014,
   5918,
   'Callanthias_australi